In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.2 MB/s eta 0:00:00


In [39]:
import os
import cv2
import numpy as np
import joblib
import torch

from skimage.feature import hog
from ultralytics import YOLO
import torchvision.transforms.functional as F


In [40]:
# ===== CLASSIFIER =====
XGB_MODEL_PATH = "/content/drive/MyDrive/RF_Val/xgboost_model.json"
LABEL_PATH = "/content/drive/MyDrive/RF_Val/label_classes.npy"

# ===== DETECTORS =====
FASTER_RCNN_PATH = "/content/drive/MyDrive/fasterrcnn_vivax_aug.pth"
RTDETR_PATH = "/content/drive/MyDrive/Vivax_data_augmented/vivax_results/train/weights/best.pt"
YOLOV11_PATH = "/content/drive/MyDrive/weights_vivax_yolo/best.pt"

# ===== DATA =====
VAL_DIR = "/content/drive/MyDrive/Ovale/images/valid"

# ===== OUTPUT =====
OUT_FRCNN = "/content/drive/MyDrive/xgbst_fastercnn"
OUT_RTDETR = "/content/drive/MyDrive/xgbst_rtdetr"
OUT_YOLO = "/content/drive/MyDrive/xgbst_yolo"

for d in [OUT_FRCNN, OUT_RTDETR, OUT_YOLO]:
    os.makedirs(d, exist_ok=True)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
IMG_SIZE = 64
TARGET_IMG_SIZE = 640 # New constant for consistent image resizing
VALID_EXT = (".png", ".jpg", ".jpeg")

In [41]:
CLASS_NAMES = list(np.load(LABEL_PATH))

STAGE_CLASSES = {
    1: "Ring",
    2: "Trophozoite",
    3: "Schizont",
    4: "Gametocyte"
}

In [42]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier()
xgb_model.load_model(XGB_MODEL_PATH)

print("✅ XGBoost classifier loaded")

✅ XGBoost classifier loaded


In [43]:
def xgb_preprocess(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError(f"Cannot read image: {image_path}")

    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

    features = hog(
        img,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        block_norm="L2-Hys"
    )
    return features.reshape(1, -1)

In [44]:
def classify_species_xgb(image_path):
    features = xgb_preprocess(image_path)
    idx = int(xgb_model.predict(features)[0])
    confidence = float(xgb_model.predict_proba(features).max())
    return CLASS_NAMES[idx], confidence

In [45]:
def draw_species_label(img, species, conf):
    label = f"{species} ({conf:.2f})"
    font, scale, thick = cv2.FONT_HERSHEY_SIMPLEX, 0.9, 3
    (w, h), _ = cv2.getTextSize(label, font, scale, thick)

    cv2.rectangle(img, (10,10), (10+w+14,10+h+20), (0,255,0), -1)
    cv2.putText(img, label, (17,10+h+10), font, scale, (0,0,0), thick, cv2.LINE_AA)
    return img

In [46]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def load_frcnn(path):
    model = fasterrcnn_resnet50_fpn(weights=None)
    in_feat = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_feat, 5)
    model.load_state_dict(torch.load(path, map_location=DEVICE))
    model.to(DEVICE).eval()
    print("✅ Faster R-CNN loaded")
    return model

faster_rcnn = load_frcnn(FASTER_RCNN_PATH)

✅ Faster R-CNN loaded


In [47]:
from ultralytics import YOLO
rtdetr_model = YOLO(RTDETR_PATH)
print("✅ RT-DETR loaded")

✅ RT-DETR loaded


In [48]:
from ultralytics import YOLO
yolov11 = YOLO(YOLOV11_PATH)
print("✅ YOLOv11 loaded")

✅ YOLOv11 loaded


In [49]:
def draw_boxes(img, boxes, labels, scores, label_map, thresh=0.5):
    for b, l, s in zip(boxes, labels, scores):
        if s < thresh or l not in label_map:
            continue
        x1,y1,x2,y2 = map(int, b)
        txt = f"{label_map[l]} ({s:.2f})"

        cv2.rectangle(img,(x1,y1),(x2,y2),(0,0,255),3)
        (w,h),_ = cv2.getTextSize(txt, cv2.FONT_HERSHEY_SIMPLEX,0.6,2)
        cv2.rectangle(img,(x1,y1-h-10),(x1+w+6,y1),(0,0,255),-1)
        cv2.putText(img,txt,(x1+3,y1-4),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,255,255),2)
    return img

In [57]:
def run_xgb_frcnn(image_path):
    img = cv2.imread(image_path)
    species, conf = classify_species_xgb(image_path)
    img = draw_species_label(img, species, conf)

    if species != "Uninfected":
        # Convert to tensor and ensure it's FloatTensor to match model weights
        tensor = F.to_tensor(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)).to(DEVICE)
        with torch.no_grad():
            out = faster_rcnn([tensor])[0]
        img = draw_boxes(
            img,
            out["boxes"].cpu().numpy(),
            out["labels"].cpu().numpy(),
            out["scores"].cpu().numpy(),
            STAGE_CLASSES
        )

    cv2.imwrite(os.path.join(OUT_FRCNN, "frcnn_"+os.path.basename(image_path)), img)

In [66]:
def run_xgb_rtdetr(image_path):
    img = cv2.imread(image_path)
    species, conf = classify_species_xgb(image_path)
    img = draw_species_label(img, species, conf)

    if species != "Uninfected":
        # Resize image for consistent input to the model
        img_resized = cv2.resize(img, (TARGET_IMG_SIZE, TARGET_IMG_SIZE))
        # Convert resized image (NumPy array, BGR) to PyTorch Tensor (RGB) and add batch dimension
        img_rgb_tensor = F.to_tensor(cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)).unsqueeze(0).to(DEVICE)
        results = rtdetr_model(img_rgb_tensor) # Pass the tensor to the model
        for r in results:
            img = draw_boxes(
                img,
                r.boxes.xyxy.cpu().numpy(),
                r.boxes.cls.cpu().numpy().astype(int) + 1, # assuming 0-indexed labels, convert to 1-indexed for STAGE_CLASSES
                r.boxes.conf.cpu().numpy(),
                STAGE_CLASSES
            )

    cv2.imwrite(os.path.join(OUT_RTDETR, "rtdetr_"+os.path.basename(image_path)), img)

In [67]:
def run_xgb_yolo(image_path):
    img = cv2.imread(image_path)
    species, conf = classify_species_xgb(image_path)
    img = draw_species_label(img, species, conf)

    if species != "Uninfected":
        # Resize image for consistent input to the model
        img_resized = cv2.resize(img, (TARGET_IMG_SIZE, TARGET_IMG_SIZE))
        # Convert resized image (NumPy array, BGR) to PyTorch Tensor (RGB) and add batch dimension
        img_rgb_tensor = F.to_tensor(cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)).unsqueeze(0).to(DEVICE)
        results = yolov11(img_rgb_tensor) # Pass the tensor to the model
        for r in results:
            img = draw_boxes(
                img,
                r.boxes.xyxy.cpu().numpy(),
                r.boxes.cls.cpu().numpy().astype(int)+1,
                r.boxes.conf.cpu().numpy(),
                STAGE_CLASSES
            )

    cv2.imwrite(os.path.join(OUT_YOLO, "yolo_"+os.path.basename(image_path)), img)

In [68]:
import torchvision.transforms.functional as F

TARGET_IMG_SIZE = 640 # Ensure TARGET_IMG_SIZE is defined here

for img_name in os.listdir(VAL_DIR):
    if not img_name.lower().endswith(VALID_EXT):
        continue

    path = os.path.join(VAL_DIR, img_name)

    run_xgb_frcnn(path)
    run_xgb_rtdetr(path)
    run_xgb_yolo(path)


0: 640x640 3 Trophozoites, 73.3ms
Speed: 0.0ms preprocess, 73.3ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Schizont, 12.9ms
Speed: 0.0ms preprocess, 12.9ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 5 Trophozoites, 57.9ms
Speed: 0.0ms preprocess, 57.9ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Ring, 10.6ms
Speed: 0.0ms preprocess, 10.6ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 Schizonts, 1 Trophozoite, 52.7ms
Speed: 0.0ms preprocess, 52.7ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Schizont, 16.7ms
Speed: 0.0ms preprocess, 16.7ms inference, 2.6ms postprocess per image at shape (1, 3, 640, 640)
